In [1]:
import cudf
import os

In [2]:
os.environ['CUDA_VISIBLE_DEVICES'] = "0"

In [3]:
df = cudf.read_parquet('../data/all_states_us_lat_long_sex.parquet/*')[['GISJOIN', 'sex']]

In [4]:
df['GISJOIN_og'] = df.GISJOIN
df['GISJOIN'] = (df.GISJOIN/1000).astype('int')

In [5]:
df

,GISJOIN,sex,GISJOIN_og
900512,1000100201001,0,1000100201001000
900513,1000100201001,0,1000100201001000
900514,1000100201001,0,1000100201001000
900515,1000100201001,0,1000100201001000
900516,1000100201001,0,1000100201001000
...,...,...,...
308744617,56004509513003,1,56004509513003128
308744618,56004509513003,1,56004509513003128
308744619,56004509513003,1,56004509513003128
308744620,56004509513003,1,56004509513003128


In [6]:
df.dtypes

GISJOIN       int64
sex            int8
GISJOIN_og    int64
dtype: object

## Adding education block wise - Male & Female

<b>JN9E001</b>:     Total</br>
<b>JN9E002</b>:     Male</br>
<b>JN9E003</b>:     Male: No schooling completed</br>
<b>JN9E004</b>:     Male: Nursery to 4th grade</br>
<b>JN9E005</b>:     Male: 5th and 6th grade</br>
<b>JN9E006</b>:     Male: 7th and 8th grade</br>
<b>JN9E007</b>:     Male: 9th grade</br>
<b>JN9E008</b>:     Male: 10th grade</br>
<b>JN9E009</b>:     Male: 11th grade</br>
<b>JN9E010</b>:     Male: 12th grade, no diploma</br>
<b>JN9E011</b>:     Male: High school graduate, GED, or alternative</br>
<b>JN9E012</b>:     Male: Some college, less than 1 year</br>
<b>JN9E013</b>:     Male: Some college, 1 or more years, no degree</br>
<b>JN9E014</b>:     Male: Associate's degree</br>
<b>JN9E015</b>:     Male: Bachelor's degree</br>
<b>JN9E016</b>:     Male: Master's degree</br>
<b>JN9E017</b>:     Male: Professional school degree</br>
<b>JN9E018</b>:     Male: Doctorate degree</br>
<b>JN9E019</b>:     Female</br>
<b>JN9E020</b>:     Female: No schooling completed</br>
<b>JN9E021</b>:     Female: Nursery to 4th grade</br>
<b>JN9E022</b>:     Female: 5th and 6th grade</br>
<b>JN9E023</b>:     Female: 7th and 8th grade</br>
<b>JN9E024</b>:     Female: 9th grade</br>
<b>JN9E025</b>:     Female: 10th grade</br>
<b>JN9E026</b>:     Female: 11th grade</br>
<b>JN9E027</b>:     Female: 12th grade, no diploma</br>
<b>JN9E028</b>:     Female: High school graduate, GED, or alternative</br>
<b>JN9E029</b>:     Female: Some college, less than 1 year</br>
<b>JN9E030</b>:     Female: Some college, 1 or more years, no degree</br>
<b>JN9E031</b>:     Female: Associate's degree</br>
<b>JN9E032</b>:     Female: Bachelor's degree</br>
<b>JN9E033</b>:     Female: Master's degree</br>
<b>JN9E034</b>:     Female: Professional school degree</br>
<b>JN9E035</b>:     Female: Doctorate degree</br>

In [47]:
df_cat = cudf.read_csv('/home/ajay/data/census/all_states_block_group/nhgis0016_csv/nhgis0016_ds176_20105_2010_blck_grp.csv')

df_cat.STATEA = df_cat.STATEA.astype('int')
df_cat = df_cat.query('STATEA != 72')

df_cat.GISJOIN = df_cat.GISJOIN.str.replace('G', '').astype('int')

sex_by_education_cols = ['JN9E002','JN9E003','JN9E004','JN9E005','JN9E006','JN9E007','JN9E008','JN9E009','JN9E010','JN9E011','JN9E012','JN9E013','JN9E014','JN9E015','JN9E016','JN9E017','JN9E018','JN9E019','JN9E020','JN9E021','JN9E022','JN9E023','JN9E024','JN9E025','JN9E026','JN9E027','JN9E028','JN9E029','JN9E030','JN9E031','JN9E032','JN9E033','JN9E034','JN9E035']

sex_by_education_cols_male = ['JN9E002','JN9E003','JN9E004','JN9E005','JN9E006','JN9E007','JN9E008','JN9E009','JN9E010','JN9E011','JN9E012','JN9E013','JN9E014','JN9E015','JN9E016','JN9E017','JN9E018']
sex_by_education_cols_female = ['JN9E019','JN9E020','JN9E021','JN9E022','JN9E023','JN9E024','JN9E025','JN9E026','JN9E027','JN9E028','JN9E029','JN9E030','JN9E031','JN9E032','JN9E033','JN9E034','JN9E035']

sex_by_education_cols_without_total = ['JN9E003','JN9E004','JN9E005','JN9E006','JN9E007','JN9E008','JN9E009','JN9E010','JN9E011','JN9E012','JN9E013','JN9E014','JN9E015','JN9E016','JN9E017','JN9E018','JN9E020','JN9E021','JN9E022','JN9E023','JN9E024','JN9E025','JN9E026','JN9E027','JN9E028','JN9E029','JN9E030','JN9E031','JN9E032','JN9E033','JN9E034','JN9E035']
df_cat = df_cat[['GISJOIN']+sex_by_education_cols]

for i in df_cat.columns:
    if i != 'GISJOIN':
        df_cat[i] = df_cat[i].astype('int16')

In [48]:
df_cat

,GISJOIN,JN9E002,JN9E003,JN9E004,JN9E005,JN9E006,JN9E007,JN9E008,JN9E009,JN9E010,...,JN9E026,JN9E027,JN9E028,JN9E029,JN9E030,JN9E031,JN9E032,JN9E033,JN9E034,JN9E035
0,1000100201001,184,11,0,0,0,11,0,0,0,...,0,0,65,27,45,7,20,73,0,0
1,1000100201002,427,0,7,6,27,0,21,18,23,...,23,0,154,0,55,52,61,0,0,0
2,1000100202001,316,13,12,0,57,0,0,14,0,...,50,11,172,0,82,11,53,19,15,0
3,1000100202002,246,0,0,0,0,0,23,10,0,...,45,0,62,7,0,31,33,0,0,8
4,1000100203001,786,0,0,0,11,0,37,9,16,...,18,52,353,46,117,62,138,23,7,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
217735,56004509511001,441,0,0,0,16,5,9,20,0,...,17,0,179,41,108,56,54,6,0,0
217736,56004509511002,741,0,0,0,29,14,9,26,0,...,32,10,212,45,74,62,60,45,0,0
217737,56004509513001,410,0,0,0,28,8,9,10,11,...,0,8,244,13,51,24,50,0,0,7
217738,56004509513002,414,0,0,0,0,0,31,13,17,...,10,0,128,85,63,17,99,2,0,0


In [49]:
all(df_cat[sex_by_education_cols_male[1:]].to_pandas().sum(axis=1).values == df_cat.JN9E002.to_array())

True

In [50]:
all(df_cat[sex_by_education_cols_female[1:]].to_pandas().sum(axis=1).values == df_cat.JN9E019.to_array())

True

In [51]:
for i in sex_by_education_cols_male:
    if i != 'JN9E002':
        df_cat[i] = df_cat[i]/df_cat['JN9E002'].astype('float32')#*100).floor().astype('int16')

for i in sex_by_education_cols_female:
    if i != 'JN9E019':
        df_cat[i] = df_cat[i]/df_cat['JN9E019'].astype('float32')#*100).floor().astype('int16')
        
df_cat.drop_column('JN9E002')
df_cat.drop_column('JN9E019')

In [52]:
df_cat

,GISJOIN,JN9E003,JN9E004,JN9E005,JN9E006,JN9E007,JN9E008,JN9E009,JN9E010,JN9E011,...,JN9E026,JN9E027,JN9E028,JN9E029,JN9E030,JN9E031,JN9E032,JN9E033,JN9E034,JN9E035
0,1000100201001,0.059783,0.000000,0.000000,0.000000,0.059783,0.000000,0.000000,0.000000,0.467391,...,0.000000,0.000000,0.262097,0.108871,0.181452,0.028226,0.080645,0.294355,0.000000,0.000000
1,1000100201002,0.000000,0.016393,0.014052,0.063232,0.000000,0.049180,0.042155,0.053864,0.306792,...,0.060052,0.000000,0.402089,0.000000,0.143603,0.135770,0.159269,0.000000,0.000000,0.000000
2,1000100202001,0.041139,0.037975,0.000000,0.180380,0.000000,0.000000,0.044304,0.000000,0.199367,...,0.107759,0.023707,0.370690,0.000000,0.176724,0.023707,0.114224,0.040948,0.032328,0.000000
3,1000100202002,0.000000,0.000000,0.000000,0.000000,0.000000,0.093496,0.040650,0.000000,0.126016,...,0.188285,0.000000,0.259414,0.029289,0.000000,0.129707,0.138075,0.000000,0.000000,0.033473
4,1000100203001,0.000000,0.000000,0.000000,0.013995,0.000000,0.047074,0.011450,0.020356,0.258270,...,0.019956,0.057650,0.391353,0.050998,0.129712,0.068736,0.152993,0.025499,0.007761,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
217735,56004509511001,0.000000,0.000000,0.000000,0.036281,0.011338,0.020408,0.045351,0.000000,0.399093,...,0.036170,0.000000,0.380851,0.087234,0.229787,0.119149,0.114894,0.012766,0.000000,0.000000
217736,56004509511002,0.000000,0.000000,0.000000,0.039136,0.018893,0.012146,0.035088,0.000000,0.437247,...,0.057451,0.017953,0.380610,0.080790,0.132855,0.111311,0.107720,0.080790,0.000000,0.000000
217737,56004509513001,0.000000,0.000000,0.000000,0.068293,0.019512,0.021951,0.024390,0.026829,0.378049,...,0.000000,0.019560,0.596577,0.031785,0.124694,0.058680,0.122249,0.000000,0.000000,0.017115
217738,56004509513002,0.000000,0.000000,0.000000,0.000000,0.000000,0.074879,0.031401,0.041063,0.454106,...,0.023364,0.000000,0.299065,0.198598,0.147196,0.039720,0.231308,0.004673,0.000000,0.000000


### merge df_cat's index with df

In [53]:
df = df.merge(df_cat[['GISJOIN']], on='GISJOIN')

In [54]:
df

,GISJOIN,sex,GISJOIN_og
0,1000100205001,0,1000100205001004
1,1000100205001,0,1000100205001004
2,1000100205001,0,1000100205001004
3,1000100205001,0,1000100205001004
4,1000100205001,0,1000100205001004
...,...,...,...
308745304,56004509511001,0,56004509511001160
308745305,56004509511001,0,56004509511001160
308745306,56004300002001,0,56004300002001000
308745307,56004300002001,0,56004300002001000


#### Make sure total ratio is a complete 1 (or 0 incase there are no values)

In [55]:
df1 = df.groupby(by=['GISJOIN', 'sex'], as_index=False).count()

In [56]:
df1 = df1.one_hot_encoding('sex', 'gender', [0,1])

In [57]:
df1.drop_column('sex')

In [58]:
df1.gender_0 = (df1.GISJOIN_og*df1.gender_0).astype('int')
df1.gender_1 = (df1.GISJOIN_og*df1.gender_1).astype('int')

In [59]:
df1.drop_column('GISJOIN_og')

In [60]:
df1 = df1.groupby('GISJOIN', as_index=False).sum()
df1

,GISJOIN,gender_0,gender_1
0,1000100201001,297,401
1,1000100201002,641,573
2,1000100202001,393,610
3,1000100202002,605,562
4,1000100203001,1099,1450
...,...,...,...
216859,56004509511001,822,690
216860,56004509511002,1040,762
216861,56004509513001,556,620
216862,56004509513002,597,711


#### Merge population in df with ratios in df_cat

In [61]:
df1 = df1.merge(df_cat, on='GISJOIN')

In [62]:
df1

,GISJOIN,gender_0,gender_1,JN9E003,JN9E004,JN9E005,JN9E006,JN9E007,JN9E008,JN9E009,...,JN9E026,JN9E027,JN9E028,JN9E029,JN9E030,JN9E031,JN9E032,JN9E033,JN9E034,JN9E035
0,11000100099011,286,431,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.326861,0.074434,0.093851,0.210356,0.229773,0.022654,0.000000,0.000000
1,11000100099012,702,830,0.000000,0.0,0.015296,0.013384,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.167183,0.054180,0.199690,0.000000,0.218266,0.202786,0.095975,0.029412
2,11000100099021,482,641,0.000000,0.0,0.000000,0.031496,0.000000,0.000000,0.000000,...,0.044968,0.023555,0.250535,0.051392,0.117773,0.000000,0.203426,0.098501,0.021413,0.000000
3,11000100099022,696,1086,0.000000,0.0,0.000000,0.031835,0.000000,0.020599,0.142322,...,0.067174,0.000000,0.333333,0.036755,0.073511,0.122940,0.147022,0.143219,0.000000,0.000000
4,11000100099031,524,452,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.204301,0.064516,0.397849,0.059140,0.053763,0.000000,0.220430,0.000000,0.000000,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
216859,55000709603003,598,651,0.000000,0.0,0.000000,0.006479,0.000000,0.015119,0.000000,...,0.000000,0.008834,0.287986,0.026502,0.047703,0.226148,0.273852,0.061837,0.031802,0.010601
216860,55000709604001,757,738,0.005190,0.0,0.000000,0.022491,0.010381,0.013841,0.013841,...,0.012433,0.000000,0.330373,0.060391,0.241563,0.134991,0.136767,0.031972,0.007105,0.000000
216861,55000709604002,375,355,0.000000,0.0,0.016807,0.000000,0.012605,0.012605,0.000000,...,0.008734,0.013100,0.475983,0.021834,0.161572,0.082969,0.082969,0.104803,0.017467,0.000000
216862,55000709604003,464,407,0.003205,0.0,0.000000,0.054487,0.000000,0.051282,0.016026,...,0.027439,0.000000,0.295732,0.033537,0.234756,0.146341,0.164634,0.033537,0.000000,0.000000


In [63]:
#males
df1.to_pandas().query('JN9E003.isnull()')

,GISJOIN,gender_0,gender_1,JN9E003,JN9E004,JN9E005,JN9E006,JN9E007,JN9E008,JN9E009,...,JN9E026,JN9E027,JN9E028,JN9E029,JN9E030,JN9E031,JN9E032,JN9E033,JN9E034,JN9E035
748,12007100401153,1212,1632,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
858,5011900032051,0,2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
872,6003705041021,0,22,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1747,6003705409024,0,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2294,2006000001002,0,16,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
214475,54006100101025,325,398,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
215253,55002500011011,1008,1334,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0.0,0.0,0.857143,0.0,0.0,0.0,0.142857,0.0,0.0,0.0
215255,55002500011021,378,406,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0.0,0.0,0.000000,1.0,0.0,0.0,0.000000,0.0,0.0,0.0
215256,55002500011022,1178,1188,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [64]:
#females
df1.to_pandas().query('JN9E020.isnull()')

,GISJOIN,gender_0,gender_1,JN9E003,JN9E004,JN9E005,JN9E006,JN9E007,JN9E008,JN9E009,...,JN9E026,JN9E027,JN9E028,JN9E029,JN9E030,JN9E031,JN9E032,JN9E033,JN9E034,JN9E035
748,12007100401153,1212,1632,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
858,5011900032051,0,2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
872,6003705041021,0,22,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1747,6003705409024,0,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2294,2006000001002,0,16,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
214475,54006100101025,325,398,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
214683,55002500016063,466,469,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
215254,55002500011012,930,925,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
215256,55002500011022,1178,1188,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [65]:
df2 = df1.copy()

In [66]:
#males
for i in sex_by_education_cols_male[1:]:
    df1[i] = ((df1[i] * df1.gender_0)).floor().astype('int16')
    df2[i] = ((df2[i] * df2.gender_0) - df1[i]).astype('float32')
    
t = df2.to_pandas().query('JN9E003.isnull()').index #everything is null for a row in this case

for i in sex_by_education_cols_male[1:]:
    df1[i].loc[[t]] = 0
    df2[i].loc[[t]] = (4.55/100)

df1['diff_male'] = df1.gender_0.to_pandas() - df1[sex_by_education_cols_male[1:]].to_pandas().sum(axis=1)

In [67]:
#females
# df2 = df1.copy()
for i in sex_by_education_cols_female[1:]:
    df1[i] = ((df1[i] * df1.gender_1)).floor().astype('int16')
    df2[i] = ((df2[i] * df2.gender_1) - df1[i]).astype('float32')
    
t = df2.to_pandas().query('JN9E020.isnull()').index #everything is null for a row in this case

for i in sex_by_education_cols_female[1:]:
    df1[i].loc[[t]] = 0
    df2[i].loc[[t]] = (4.55/100)

df1['diff_female'] = df1.gender_1.to_pandas() - df1[sex_by_education_cols_female[1:]].to_pandas().sum(axis=1)

In [68]:
df1

,GISJOIN,gender_0,gender_1,JN9E003,JN9E004,JN9E005,JN9E006,JN9E007,JN9E008,JN9E009,...,JN9E028,JN9E029,JN9E030,JN9E031,JN9E032,JN9E033,JN9E034,JN9E035,diff_male,diff_female
0,11000100099011,286,431,0,0,0,0,0,0,0,...,140,32,40,90,99,9,0,0,2,3
1,11000100099012,702,830,0,0,10,9,0,0,0,...,138,44,165,0,181,168,79,24,6,5
2,11000100099021,482,641,0,0,0,15,0,0,0,...,160,32,75,0,130,63,13,0,4,5
3,11000100099022,696,1086,0,0,0,22,0,14,99,...,362,39,79,133,159,155,0,0,4,6
4,11000100099031,524,452,0,0,0,0,0,0,0,...,179,26,24,0,99,0,0,0,1,3
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
216859,55000709603003,598,651,0,0,0,3,0,9,0,...,187,17,31,147,178,40,20,6,5,5
216860,55000709604001,757,738,3,0,0,17,7,10,10,...,243,44,178,99,100,23,5,0,6,7
216861,55000709604002,375,355,0,0,6,0,4,4,0,...,168,7,57,29,29,37,6,0,6,5
216862,55000709604003,464,407,1,0,0,25,0,23,7,...,120,13,95,59,67,13,0,0,6,4


In [69]:
df1.diff_male.max()

2475

In [70]:
df2[sex_by_education_cols_male[1:]].to_pandas().sum(axis=1).unique()

array([2.00000426, 6.00000921, 4.00000181, ..., 6.99999225, 5.99998979,
       5.00000223])

In [71]:
print(df1.diff_male.sum() == (df1.gender_0.to_array().sum() - df1[sex_by_education_cols_male[1:]].to_pandas().sum(axis=1).values.sum()))
print(df1.diff_female.sum() == (df1.gender_1.to_array().sum() - df1[sex_by_education_cols_female[1:]].to_pandas().sum(axis=1).values.sum()))

True
True


In [72]:
print(df1.gender_0.to_array().sum() == df1[sex_by_education_cols_male[1:]].to_pandas().sum(axis=1).values.sum())
print(df1.gender_1.to_array().sum() == df1[sex_by_education_cols_female[1:]].to_pandas().sum(axis=1).values.sum())

False
False


#### filling the difference

Since they don't add up to the total population, lets apply the largest remainder method, mentioned here -> https://revs.runtime-revolution.com/getting-100-with-rounded-percentages-273ffa70252b

In [73]:
import cupy

cp_array_male = cupy.array(df2[sex_by_education_cols_male[1:]].to_gpu_matrix())
cp_array_male = cupy.nan_to_num(cp_array_male/ cp_array_male.sum(axis=1, keepdims=True))
cp_array_male_sorted = cupy.asnumpy(cp_array_male.argsort())
cp_array_male = cupy.asnumpy(cp_array_male)

cp_array_female = cupy.array(df2[sex_by_education_cols_female[1:]].to_gpu_matrix())
cp_array_female = cupy.nan_to_num(cp_array_female/ cp_array_female.sum(axis=1, keepdims=True))
cp_array_female_sorted = cupy.asnumpy(cp_array_female.argsort())
cp_array_female = cupy.asnumpy(cp_array_female)

Handle remaining remainder values

In [74]:
from multiprocessing import  Pool
from functools import partial
import numpy as np
import pandas as pd

def parallelize(data, func, num_of_processes=8):
    data_split = np.array_split(data, num_of_processes)
    pool = Pool(num_of_processes)
    data = pd.concat(pool.map(func, data_split))
    pool.close()
    pool.join()
    return data

def run_on_subset(func, data_subset):
    return data_subset.apply(func, axis=1)

def parallelize_on_rows(data, func, num_of_processes=8):
    return parallelize(data, partial(run_on_subset, func), num_of_processes)

In [76]:
df1_pandas = df1.to_pandas()

count = 0

def applyrows(row):
    global count, cp_array_male_sorted, cp_array_male, cp_array_female_sorted, cp_array_female
    #males
    sorted_series_male = cp_array_male_sorted[row.name][::-1]
    i = row['diff_male']
    while i > 0:
        for sorted_index in sorted_series_male:
            sorted_val = cp_array_male[row.name][sorted_index]
            if sorted_val == 0:
                break
            if int(sorted_val * i) <= 1:
                t_value = i
            else:
                t_value = int(sorted_val * i)
            t_index = 'JN9E0'+str(sorted_index+3).zfill(2)
            row[t_index] += t_value
            i -= t_value
    #females
    sorted_series_female = cp_array_female_sorted[row.name][::-1]
    i = row['diff_female']
    while i > 0:
        for sorted_index in sorted_series_female:
            sorted_val = cp_array_female[row.name][sorted_index]
            if sorted_val == 0:
                break
            if int(sorted_val * i) <= 1:
                t_value = i
            else:
                t_value = int(sorted_val * i)
            t_index = 'JN9E0'+str(sorted_index+20).zfill(2)
            row[t_index] += t_value
            i -= t_value
    
    count += 1
    if count%10000 == 0: 
        print('                                  ', end='\r')
        print('count', count,row.name, end='\r')
    return row

# df2_pandas = df1_pandas.apply(applyrows, axis=1)
df2_pandas = parallelize_on_rows(df1_pandas, applyrows) 

In [77]:
df2_pandas

,GISJOIN,gender_0,gender_1,JN9E003,JN9E004,JN9E005,JN9E006,JN9E007,JN9E008,JN9E009,...,JN9E028,JN9E029,JN9E030,JN9E031,JN9E032,JN9E033,JN9E034,JN9E035,diff_male,diff_female
0,11000100099011,286,431,0,0,0,0,0,0,0,...,143,32,40,90,99,9,0,0,2,3
1,11000100099012,702,830,0,0,10,9,0,0,0,...,138,49,165,0,181,168,79,24,6,5
2,11000100099021,482,641,0,0,0,15,0,0,0,...,160,37,75,0,130,63,13,0,4,5
3,11000100099022,696,1086,0,0,0,22,0,14,99,...,362,39,79,133,159,155,0,0,4,6
4,11000100099031,524,452,0,0,0,0,0,0,0,...,182,26,24,0,99,0,0,0,1,3
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
216859,55000709603003,598,651,0,0,0,8,0,9,0,...,187,17,31,147,178,40,20,11,5,5
216860,55000709604001,757,738,9,0,0,17,7,10,10,...,243,44,178,99,100,23,5,0,6,7
216861,55000709604002,375,355,0,0,6,0,4,4,0,...,173,7,57,29,29,37,6,0,6,5
216862,55000709604003,464,407,1,0,0,25,0,23,7,...,120,13,95,59,67,17,0,0,6,4


In [78]:
print(df2_pandas.gender_0.values.sum() == df2_pandas[sex_by_education_cols_male[1:]].sum(axis=1).values.sum())
print(df2_pandas.gender_1.values.sum() == df2_pandas[sex_by_education_cols_female[1:]].sum(axis=1).values.sum())

True
True


In [79]:
df_final = cudf.from_pandas(df2_pandas)

In [80]:
df_final

,GISJOIN,gender_0,gender_1,JN9E003,JN9E004,JN9E005,JN9E006,JN9E007,JN9E008,JN9E009,...,JN9E028,JN9E029,JN9E030,JN9E031,JN9E032,JN9E033,JN9E034,JN9E035,diff_male,diff_female
0,11000100099011,286,431,0,0,0,0,0,0,0,...,143,32,40,90,99,9,0,0,2,3
1,11000100099012,702,830,0,0,10,9,0,0,0,...,138,49,165,0,181,168,79,24,6,5
2,11000100099021,482,641,0,0,0,15,0,0,0,...,160,37,75,0,130,63,13,0,4,5
3,11000100099022,696,1086,0,0,0,22,0,14,99,...,362,39,79,133,159,155,0,0,4,6
4,11000100099031,524,452,0,0,0,0,0,0,0,...,182,26,24,0,99,0,0,0,1,3
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
216859,55000709603003,598,651,0,0,0,8,0,9,0,...,187,17,31,147,178,40,20,11,5,5
216860,55000709604001,757,738,9,0,0,17,7,10,10,...,243,44,178,99,100,23,5,0,6,7
216861,55000709604002,375,355,0,0,6,0,4,4,0,...,173,7,57,29,29,37,6,0,6,5
216862,55000709604003,464,407,1,0,0,25,0,23,7,...,120,13,95,59,67,17,0,0,6,4


In [81]:
print(df_final.gender_0.to_array().sum() == df_final[sex_by_education_cols_male[1:]].to_pandas().sum(axis=1).values.sum())
print(df_final.gender_1.to_array().sum() == df_final[sex_by_education_cols_female[1:]].to_pandas().sum(axis=1).values.sum())

True
True


In [82]:
df_final.drop_column('diff_male')
df_final.drop_column('diff_female')
# df_final.drop_column('GISJOIN')
# df_final.rename({'GISJOIN_og':'GISJOIN'}, inplace=True)

In [83]:
df_final.to_parquet('education_step_1_final.parquet')

/home/ajay/anaconda3/envs/rapids_dev_0.12/lib/python3.6/site-packages/cudf/io/parquet.py:70: UserWarning: Using CPU via PyArrow to write Parquet dataset, this will be GPU accelerated in the future
  "Using CPU via PyArrow to write Parquet dataset, this will "


In [84]:
df.to_parquet('GISJOIN_to_GISJOIN_og_mapping.parquet')